In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
import sys,os
import json
import random
import altair as alt

from scipy import stats
from pymongo import MongoClient
from tqdm import tqdm

In [2]:
client = MongoClient('localhost', 27017)
db = client.TFE
collection = db.news_cleaned

In [3]:
res = collection.aggregate(
    [
        { 
            "$match" : {
                "type" : {
                    "$in" : [
                        "fake", 
                        "reliable"
                    ]
                }
            }
        }, 
        { 
            "$group" : {
                "_id" : {
                    "type" : "$type"
                }, 
                "newsCount" : {
                    "$sum" : 1.0
                }
            }
        }
    ]
)

types = []
count = []
for r in res:
    types.append(r['_id']['type'])
    count.append(r['newsCount'])
    
data = pd.DataFrame({'x' : types, 'y' : count})

chart1 = alt.Chart(data).mark_bar().encode(
    x = alt.X('x', axis=alt.Axis(title="type")),
    y = alt.Y('y', axis=alt.Axis(title="Count"))
)

In [4]:
res = collection.aggregate(
    [
        { 
            "$match" : {
                "type" : {
                    "$in" : [
                        "fake", 
                        "reliable"
                    ]
                },
                'domain' : {'$nin' : ['nytimes.com', 'beforeitsnews.com']}
            }
        }, 
        { 
            "$group" : {
                "_id" : {
                    "type" : "$type"
                }, 
                "newsCount" : {
                    "$sum" : 1.0
                }
            }
        }
    ]
)

types = []
count = []
for r in res:
    types.append(r['_id']['type'])
    count.append(r['newsCount'])
    
data = pd.DataFrame({'x' : types, 'y' : count})

chart2 = alt.Chart(data).mark_bar().encode(
    x = alt.X('x', axis=alt.Axis(title="type")),
    y = alt.Y('y', axis=alt.Axis(title="Count"))
)

In [5]:
chart = chart1 | chart2
chart.save('output/news_count.png', webdriver='firefox')

# Average Sentence Length charts

In [6]:
avgSentenceLength = {}
for t in types:
    data = collection.find({'type' : t}, {'avgSentenceLength' : True, '_id' : False})
    data = [x['avgSentenceLength'] for x in data]
    avgSentenceLength[t] = data
    
data = pd.DataFrame(columns=['l', 'q1', 'q2', 'q3', 'h', 'type'])

p = []
for t in types:
    percentile = np.percentile(avgSentenceLength[t], [25, 50, 75], interpolation='linear')
    p.append({'l': percentile[0] - (percentile[1] - percentile[0]) * 1.5, 'q1' : percentile[0], 'q2' : percentile[1], 'q3' : percentile[2], 'h' : percentile[2] + (percentile[2] - percentile[1]) * 1.5,  'type' : t})

data = data.append(p)

base = alt.Chart(data)

lower_plot = base.mark_rule().encode(
    y=alt.Y('l', axis=alt.Axis(title="Average Sentence Length")),
    y2='q1',
    x='type'
)

middle_plot = base.mark_bar(size=5.0).encode(
    y='q1',
    y2='q3',
    x='type'
)
upper_plot = base.mark_rule().encode(
    y='h',
    y2='q3',
    x='type'
)

middle_tick = base.mark_tick(
    color='white',
    size=5.0
).encode(
    y='q2',
    x='type',
)

chart3 = lower_plot + middle_plot + upper_plot + middle_tick

In [7]:
avgSentenceLength = {}
for t in types:
    data = collection.find({'type' : t, 'domain' : {'$nin' : ['nytimes.com', 'beforeitsnews.com']}}, {'avgSentenceLength' : True, '_id' : False})
    data = [x['avgSentenceLength'] for x in data]
    avgSentenceLength[t] = data
    
data = pd.DataFrame(columns=['l', 'q1', 'q2', 'q3', 'h', 'type'])

p = []
for t in types:
    percentile = np.percentile(avgSentenceLength[t], [25, 50, 75], interpolation='linear')
    p.append({'l': percentile[0] - (percentile[1] - percentile[0]) * 1.5, 'q1' : percentile[0], 'q2' : percentile[1], 'q3' : percentile[2], 'h' : percentile[2] + (percentile[2] - percentile[1]) * 1.5,  'type' : t})

data = data.append(p)

base = alt.Chart(data)

lower_plot = base.mark_rule().encode(
    y=alt.Y('l', axis=alt.Axis(title="Average Sentence Length")),
    y2='q1',
    x='type'
)

middle_plot = base.mark_bar(size=5.0).encode(
    y='q1',
    y2='q3',
    x='type'
)
upper_plot = base.mark_rule().encode(
    y='h',
    y2='q3',
    x='type'
)

middle_tick = base.mark_tick(
    color='white',
    size=5.0
).encode(
    y='q2',
    x='type',
)

chart4 = lower_plot + middle_plot + upper_plot + middle_tick

In [8]:
chart = chart3 | chart4
chart.save('output/averageSentenceLength.png', webdriver='firefox')

# Number Of Sentences

In [9]:
numSentences = {}
for t in types:
    data = collection.find({'type' : t}, {'numSentences' : True, '_id' : False})
    data = [x['numSentences'] for x in data]
    numSentences[t] = data
    
p = []
for t in types:
    percentile = np.percentile(numSentences[t], [25, 50, 75], interpolation='linear')
    p.append({'l': percentile[0] - (percentile[1] - percentile[0]) * 1.5, 'q1' : percentile[0], 'q2' : percentile[1], 'q3' : percentile[2], 'h' : percentile[2] + (percentile[2] - percentile[1]) * 1.5,  'type' : t})

data = pd.DataFrame(columns=['l', 'q1', 'q2', 'q3', 'h', 'type'])
data = data.append(p)

base = alt.Chart(data)

lower_plot = base.mark_rule().encode(
    y=alt.Y('l', axis=alt.Axis(title="Number of sentences")),
    y2='q1',
    x='type'
)

middle_plot = base.mark_bar(size=5.0).encode(
    y='q1',
    y2='q3',
    x='type'
)
upper_plot = base.mark_rule().encode(
    y='h',
    y2='q3',
    x='type'
)

middle_tick = base.mark_tick(
    color='white',
    size=5.0
).encode(
    y='q2',
    x='type',
)

chart5 = lower_plot + middle_plot + upper_plot + middle_tick

In [10]:
numSentences = {}
for t in types:
    data = collection.find({'type' : t, 'domain' : {'$nin' : ['nytimes.com', 'beforeitsnews.com']}}, {'numSentences' : True, '_id' : False})
    data = [x['numSentences'] for x in data]
    numSentences[t] = data
    
p = []
for t in types:
    percentile = np.percentile(numSentences[t], [25, 50, 75], interpolation='linear')
    p.append({'l': percentile[0] - (percentile[1] - percentile[0]) * 1.5, 'q1' : percentile[0], 'q2' : percentile[1], 'q3' : percentile[2], 'h' : percentile[2] + (percentile[2] - percentile[1]) * 1.5,  'type' : t})

data = pd.DataFrame(columns=['l', 'q1', 'q2', 'q3', 'h', 'type'])
data = data.append(p)

base = alt.Chart(data)

lower_plot = base.mark_rule().encode(
    y=alt.Y('l', axis=alt.Axis(title="Number of sentences")),
    y2='q1',
    x='type'
)

middle_plot = base.mark_bar(size=5.0).encode(
    y='q1',
    y2='q3',
    x='type'
)
upper_plot = base.mark_rule().encode(
    y='h',
    y2='q3',
    x='type'
)

middle_tick = base.mark_tick(
    color='white',
    size=5.0
).encode(
    y='q2',
    x='type',
)

chart6 = lower_plot + middle_plot + upper_plot + middle_tick

In [11]:
chart = chart5 | chart6
chart.save('output/numSentences.png', webdriver='firefox')

In [13]:
chart = chart1 | chart2 | chart3 | chart4 | chart5 | chart6
chart.save('output/numSentences.png', webdriver='firefox', scale_factor=2.0)